<a href="https://colab.research.google.com/github/abc22413/SIR-Code/blob/master/sir_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SEIR Model for COVID-19 outbreak in Singapore.
Import data for initialisation

Code adapted from: https://python.quantecon.org/sir_model.html

In [13]:
from scipy.integrate import odeint
import numpy as np

In [14]:
#Total population
#UNData: http://data.un.org/Data.aspx?q=Singapore&d=PopDiv&f=variableID%3a12%3bcrID%3a702
N = 5_850_342

#MOH: https://www.moh.gov.sg/news-highlights/details/five-more-cases-discharged-74-new-cases-of-covid-19-infection-confirmed
#Initial Infected
I = 752
#Initial Recovered
R = 245
#Initial Susceptible
S = N-I-R
#Initial Exposed
E = 2*I

In [16]:
'''
Model the trajectory of the epidemic over 182 days
'''
#Allow for Re (effective reproductive number) to be a function of time
def F(x, R_eff):
  #Disease Parameters
  recovery_period = 14
  incubation_period = 5.2
  beta = 1 / incubation_period
  delta = 1 / recovery_period

  s,e,i,r = x
  
  ds = -s*i*R_eff*delta
  de = s*i*R_eff*delta - e*beta
  di = e*beta - i*delta
  dr = i*delta
  return ds, de, di, dr

def solve(R_eff, time, x_0):
  G = lambda x, t: F(x, R_eff)
  s_new, e_new, i_new, r_new = odeint(G, x_0, time).transpose()
  return s_new, e_new, i_new, r_new

In [ ]:
#Initial conditions
s_0 = S/N
e_0 = E/N
i_0 = I/N
r_0 = R/N
x_0 = s_0, i_0, e_0, r_0

#Try different values of R_eff
trajectories = {}
#for R_eff in np.arange(0.5,6,0.5):
for R_eff in np.arange(0.5, 6, 0.5):
  #Simulate every minute
  steps_per_day = 24*60*60
  time = np.linspace(0, 182, steps_per_day)
  s_traj,e_traj,i_traj,r_traj = solve(R_eff, time, x_0)
  trajectories[R_eff]=i_traj

In [ ]:
#PLOT graph
